# Get Nightlight Data from Google Earth Engine using GEEMap

In [1]:
import ee
import geemap

## Authenticate & Initialize GEE

Requires a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and to enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. Find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [2]:
ee.Initialize()

## Create a GEEMap Object

In [3]:
m = geemap.Map(
    center=[-5, 15], 
    zoom=3, 
    basemap = 'Esri.WorldImagery'
)

## Add Layers to the Map

In [4]:
# add nightlights median
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMSLCFG
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2022-01-01', '2023-11-01'))
nighttime = dataset_night.select('avg_rad')
image_night = nighttime.median()
nighttimeVis = {'min': 0.0, 'max': 2.0}
# m.addLayer(image_night, nighttimeVis, 'Nighttime', False)


In [5]:
# add WorldPop population density layer
# https://developers.google.com/earth-engine/datasets/catalog/WorldPop_GP_100m_pop#description
dataset_pop = ee.ImageCollection('WorldPop/GP/100m/pop') \
                  .filter(ee.Filter.date('2020-01-01', '2020-12-31'))
pop = dataset_pop.select('population')
image_pop = pop.median()
popVis = {
    'min': 0.0, 
    'max': 20.0,
    'palette': ['24126c', '1fff4f', 'd4ff50'],
    'opacity': 0.5
}
# m.addLayer(image_pop, popVis, 'Population', False)


In [6]:
# overlay country boundaries with white borders
countries = ee.FeatureCollection('FAO/GAUL/2015/level0')
style = {'color': 'ffffffff', 'width': 2, 'lineType': 'solid', 'opacity': 1}
# m.addLayer(countries, style, 'Countries', False)

sl = countries.filter(ee.Filter.eq('ADM0_NAME', 'Sierra Leone'))

# note: this isn't styling the countries correctly
# the "fillColor" parameter doesn't seem to work

In [7]:
# add place names
# https://developers.google.com/earth-engine/datasets/catalog/FAO_GAUL_2015_level0
m.add_basemap('CartoDB.DarkMatterOnlyLabels')
# m.remove_layer('CartoDB.VoyagerOnlyLabels') # doesn't work

In [8]:
# display the map zoomed into Sierra Leone
m.setCenter(-11.779889, 8.460555, 8)

## Get the Landcover Data from Google Earth Engine

In [9]:
# pull in a global high resolution land cover dataset
# https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v200
landcover = ee.ImageCollection('ESA/WorldCover/v200').first()

landcover_sl = landcover.clip(sl)

visualization = {
  'bands': ['Map'],
}

# m.addLayer(landcover_sl, visualization, 'Landcover Sierra Leone', False)

# inspect this image
print(landcover_sl.getInfo())
# inspect the bands of landcover_sl
print(landcover_sl.bandNames().getInfo())
# inspect the values of the band 'Map'
print(landcover_sl.select('Map').getInfo())



{'type': 'Image', 'bands': [{'id': 'Map', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'dimensions': [4320000, 1728000], 'crs': 'EPSG:4326', 'crs_transform': [8.333333333333333e-05, 0, -180, 0, -8.333333333333333e-05, 84]}], 'version': 1699537784392512, 'id': 'ESA/WorldCover/v200/2021', 'properties': {'Map_class_names': ['Tree cover', 'Shrubland', 'Grassland', 'Cropland', 'Built-up', 'Bare / sparse vegetation', 'Snow and ice', 'Permanent water bodies', 'Herbaceous wetland', 'Mangroves', 'Moss and lichen'], 'system:time_start': 1609455600000, 'system:time_end': 1640991600000, 'Map_class_palette': ['006400', 'ffbb22', 'ffff4c', 'f096ff', 'fa0000', 'b4b4b4', 'f0f0f0', '0064c8', '0096a0', '00cf75', 'fae6a0'], 'Map_class_values': [10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 100], 'system:asset_size': 109661138988, 'system:index': '2021'}}
['Map']
{'type': 'Image', 'bands': [{'id': 'Map', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255

## Select Just for Dark Areas that are Trees and Water far away from Built Areas

In [10]:

# create a mask for the landcover
landcover_sl_built_mask = landcover_sl.eq(50)
landcover_sl_built = landcover_sl.updateMask(landcover_sl_built_mask)
# m.addLayer(landcover_sl_built, {'palette': 'red'}, 'Built')

# create a 10km buffer around built areas
built_buffer = landcover_sl_built.focal_max(10000, 'circle', 'meters')
# m.addLayer(built_buffer, {'palette': 'red'}, 'Built buffer', False)

# create a mask for the built_buffer
# need to unmask it to convert areas outside of mask from nodata to 0
rural_mask = built_buffer.eq(50).unmask(0).eq(0)
# select everywhere in Sierra Leone outside of the built buffer
landcover_sl_rural = landcover_sl.updateMask(rural_mask)
# m.addLayer(landcover_sl_rural, {'palette': 'brown'},  'Rural landcover', False)



In [11]:
# select just treed areas and water areas from landcover_sl_rural
rural_trees_mask = landcover_sl_rural.eq(10)
rural_water_mask = landcover_sl_rural.eq(80)

rural_trees = landcover_sl_rural.updateMask(rural_trees_mask)
rural_water = landcover_sl_rural.updateMask(rural_water_mask)

# add the rural tree and water layers
m.addLayer(rural_trees, {'palette': 'green'}, 'Rural tree cover', False)
m.addLayer(rural_water, {'palette': 'blue'}, 'Rural water', False)

## Sample These Areas

In [12]:
# sample 50 random points from the rural tree cover
# note this drops any points that have been masked
rural_trees_points = rural_trees.sample(
    region=sl,
    scale=1000, # 1km
    numPixels=90, # need 10000 points to get 50 that aren't masked
    seed=44,
    dropNulls=True, # drop any points that have been masked
    geometries=True
)

# convert into a feature collection of points
rural_trees_fc = ee.FeatureCollection(rural_trees_points)

m.addLayer(
    rural_trees_fc, 
    {'color': '00ff00', 'pointSize': 10}, 
    'Rural tree points', 
    True
)

rural_trees_fc

In [13]:
# repeat for rural water
rural_water_points = rural_water.sample(
    region=sl,
    scale=1000, # 1km
    numPixels=10000, # need 10000 points to get 50 that aren't masked
    seed=44,
    dropNulls=True, # drop any points that have been masked
    geometries=True
)

rural_water_fc = ee.FeatureCollection(rural_water_points)

m.addLayer(
    rural_water_fc, 
    {'color': '0000ff', 'pointSize': 10}, 
    'Rural water points', 
    True
)

rural_water_fc

In [14]:
# save the feature collections
geemap.ee_export_vector(rural_trees_fc, 'data/dark/rural_trees_points.geojson')
geemap.ee_export_vector(rural_water_fc, 'data/dark/rural_water_points.geojson')

Generating URL ...
Please wait ...
Data downloaded to /Users/ilyonsg/Documents/research/thesis/CB_Lab_Mapping_Economic_Migration/data/dark/rural_trees_points.geojson
Generating URL ...
Please wait ...
Data downloaded to /Users/ilyonsg/Documents/research/thesis/CB_Lab_Mapping_Economic_Migration/data/dark/rural_water_points.geojson


## Create a 1km Buffer Around the Sampled Points

In [15]:
# read in the geojson files
import geopandas as gpd

rural_trees_gdf = gpd.read_file('data/dark/rural_trees_points.geojson')
rural_water_gdf = gpd.read_file('data/dark/rural_water_points.geojson')

# convert gdfs to ee.FeatureCollections
rural_trees_fc = geemap.gdf_to_ee(rural_trees_gdf)
rural_water_fc = geemap.gdf_to_ee(rural_water_gdf)

In [16]:
# create a 1km buffer around the rural water points
rural_water_buffer = rural_water_fc.map(lambda f: f.buffer(1000))
rural_tree_buffer = rural_trees_fc.map(lambda f: f.buffer(1000))


## Get the Nightlights Values for each buffer

In [17]:
# get the image collectino of nightlight images
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2014-01-01', '2024-01-01'))
# select the avg_rad band
nighttime_ic = dataset_night.select('avg_rad')
# stack the images into a single image
nighttime_ic_stack = nighttime_ic.toBands()
nighttime_ic_stack

In [18]:
# apply the reduceRegions for trees
nighttime_trees_fc = nighttime_ic_stack.reduceRegions(
    collection=rural_tree_buffer, 
    reducer=ee.Reducer.mean(), 
    scale=50
)
nighttime_trees_fc

In [19]:
# apply the reduceRegions for water
nighttime_water_fc = nighttime_ic_stack.reduceRegions(
    collection=rural_water_buffer, 
    reducer=ee.Reducer.mean(), 
    scale=50
)
nighttime_water_fc

In [20]:
# get the centerpoint of each polygon feature in the feature collection
nighttime_trees_fc = nighttime_trees_fc.map(lambda f: f.set('center', f.geometry().centroid().coordinates()))
nighttime_water_fc = nighttime_water_fc.map(lambda f: f.set('center', f.geometry().centroid().coordinates()))

## Convert the Feature Collections to GeoDataFrames and Clean Data

In [21]:
# convert fc to gdf
nighttime_trees_gdf = geemap.ee_to_geopandas(nighttime_trees_fc)
nighttime_trees_gdf.head(5)

,geometry,20140101_avg_rad,20140201_avg_rad,20140301_avg_rad,20140401_avg_rad,20140501_avg_rad,20140601_avg_rad,20140701_avg_rad,20140801_avg_rad,20140901_avg_rad,...,20230401_avg_rad,20230501_avg_rad,20230601_avg_rad,20230701_avg_rad,20230801_avg_rad,20230901_avg_rad,20231001_avg_rad,Map,center,id
0,"POLYGON ((-12.17503 9.67587, -12.17758 9.67552...",-0.038235,0.138977,0.251203,0.094305,0.065352,0.094584,0.116546,0.134364,0.039151,...,0.463655,0.278641,0.379614,0.312638,0.000000,0.334841,0.000000,10,"[-12.17503067677975, 9.66687598974862]",0
1,"POLYGON ((-11.26204 8.16997, -11.26458 8.16961...",0.040457,0.061447,0.169076,0.306898,0.120875,-0.223646,0.152342,0.294298,0.129666,...,0.616264,0.346009,0.390615,0.341503,0.369959,0.428866,0.375383,10,"[-11.262036573388308, 8.160969908434629]",1
2,"POLYGON ((-11.13138 8.07272, -11.13392 8.07236...",-0.012378,0.025284,0.139634,0.081894,0.053295,-0.098647,0.120698,0.182608,0.013352,...,0.718562,0.351211,0.315646,0.441170,0.367702,0.386080,0.046991,10,"[-11.13137723485804, 8.06372341512419]",2
3,"POLYGON ((-12.94397 8.91735, -12.94652 8.91700...",-0.056811,0.018540,0.112298,0.100867,0.012564,0.091831,0.096314,0.243000,0.074746,...,0.491696,0.350578,0.294817,0.793215,0.366045,0.198691,0.771262,10,"[-12.94397360098245, 8.90835597442643]",3
4,"POLYGON ((-10.64267 8.54895, -10.64522 8.54860...",-0.034265,0.046373,0.098283,0.077979,0.037578,0.171010,0.121800,0.066459,0.083704,...,0.526977,0.317128,0.414979,0.361227,0.000000,0.387959,0.000000,10,"[-10.642673837548983, 8.5399568621107]",5


In [22]:
# convert fc to gdf
nighttime_water_gdf = geemap.ee_to_geopandas(nighttime_water_fc)
nighttime_water_gdf.head(5)

,geometry,20140101_avg_rad,20140201_avg_rad,20140301_avg_rad,20140401_avg_rad,20140501_avg_rad,20140601_avg_rad,20140701_avg_rad,20140801_avg_rad,20140901_avg_rad,...,20230401_avg_rad,20230501_avg_rad,20230601_avg_rad,20230701_avg_rad,20230801_avg_rad,20230901_avg_rad,20231001_avg_rad,Map,center,id
0,"POLYGON ((-12.66632 9.01315, -12.66887 9.01279...",-0.028725,0.024439,0.093721,0.138995,-0.014183,0.039195,0.080986,0.166109,0.066234,...,0.488067,0.389191,0.299240,0.340800,0.193704,0.558379,0.541397,80,"[-12.666320421819876, 9.004149531159928]",0
1,"POLYGON ((-13.23226 8.96823, -13.23480 8.96787...",-0.035816,0.014496,0.097973,0.070780,-0.008333,0.021782,0.114382,0.401165,0.086000,...,0.517127,0.296319,0.251019,0.000000,0.000000,0.000000,0.000000,80,"[-13.23225905083299, 8.95923376696415]",1
2,"POLYGON ((-12.79208 8.95027, -12.79463 8.94991...",-0.033231,0.064692,0.115196,0.108817,0.026321,0.044509,0.074333,0.254954,0.015511,...,0.530865,0.333067,0.291313,0.541961,0.291445,0.357026,0.497910,80,"[-12.792084561613684, 8.941267461291122]",2
3,"POLYGON ((-13.12446 8.93230, -13.12701 8.93194...",-0.058752,0.032495,0.085213,0.078138,0.019525,0.047882,0.062311,0.227329,0.138867,...,0.452269,0.308526,0.238548,0.302494,0.000000,0.000000,0.000000,80,"[-13.124461216741897, 8.923301155597281]",3
4,"POLYGON ((-13.04361 8.93230, -13.04616 8.93194...",-0.041197,0.015454,0.090730,0.096521,-0.006004,0.096218,0.120056,0.174111,0.062172,...,0.451426,0.294974,0.343269,0.504547,0.319547,0.439847,0.063976,80,"[-13.043612841166503, 8.923301155604603]",4


In [23]:
# remove "_avg_rad" from the column names that contain it
nighttime_trees_gdf.columns = nighttime_trees_gdf.columns.str.replace('_avg_rad', '')
nighttime_water_gdf.columns = nighttime_water_gdf.columns.str.replace('_avg_rad', '')
nighttime_trees_gdf.head(6)

,geometry,20140101,20140201,20140301,20140401,20140501,20140601,20140701,20140801,20140901,...,20230401,20230501,20230601,20230701,20230801,20230901,20231001,Map,center,id
0,"POLYGON ((-12.17503 9.67587, -12.17758 9.67552...",-0.038235,0.138977,0.251203,0.094305,0.065352,0.094584,0.116546,0.134364,0.039151,...,0.463655,0.278641,0.379614,0.312638,0.000000,0.334841,0.000000,10,"[-12.17503067677975, 9.66687598974862]",0
1,"POLYGON ((-11.26204 8.16997, -11.26458 8.16961...",0.040457,0.061447,0.169076,0.306898,0.120875,-0.223646,0.152342,0.294298,0.129666,...,0.616264,0.346009,0.390615,0.341503,0.369959,0.428866,0.375383,10,"[-11.262036573388308, 8.160969908434629]",1
2,"POLYGON ((-11.13138 8.07272, -11.13392 8.07236...",-0.012378,0.025284,0.139634,0.081894,0.053295,-0.098647,0.120698,0.182608,0.013352,...,0.718562,0.351211,0.315646,0.441170,0.367702,0.386080,0.046991,10,"[-11.13137723485804, 8.06372341512419]",2
3,"POLYGON ((-12.94397 8.91735, -12.94652 8.91700...",-0.056811,0.018540,0.112298,0.100867,0.012564,0.091831,0.096314,0.243000,0.074746,...,0.491696,0.350578,0.294817,0.793215,0.366045,0.198691,0.771262,10,"[-12.94397360098245, 8.90835597442643]",3
4,"POLYGON ((-10.64267 8.54895, -10.64522 8.54860...",-0.034265,0.046373,0.098283,0.077979,0.037578,0.171010,0.121800,0.066459,0.083704,...,0.526977,0.317128,0.414979,0.361227,0.000000,0.387959,0.000000,10,"[-10.642673837548983, 8.5399568621107]",5
5,"POLYGON ((-11.27817 9.64939, -11.28071 9.64903...",0.787014,1.102386,0.335038,0.113801,0.018928,0.046134,0.135542,0.214201,0.061965,...,0.467784,0.000000,0.359343,0.321327,0.277824,0.276683,0.000000,10,"[-11.278165370305556, 9.64038943419362]",6


In [24]:
# export the geodataframe to a csv
nighttime_trees_gdf.to_csv('data/dark/nighttime_trees_gdf.csv', index=False)
nighttime_water_gdf.to_csv('data/dark/nighttime_water_gdf.csv', index=False)

## Visualize the Results on a Map

In [ ]:
# map the map taller
m.layout.height = '1200px'
m

## Read in GDF, Clean, and Melt

In [33]:
import pandas as pd

# read in the csvs as geodataframes
nighttime_trees_df = pd.read_csv('data/dark/nighttime_trees_gdf.csv')
nighttime_water_df = pd.read_csv('data/dark/nighttime_water_gdf.csv')

# view head
nighttime_trees_df.head(5)


,geometry,20140101,20140201,20140301,20140401,20140501,20140601,20140701,20140801,20140901,...,20230401,20230501,20230601,20230701,20230801,20230901,20231001,Map,center,id
0,POLYGON ((-12.175030705632114 9.67587382981962...,-0.038235,0.138977,0.251203,0.094305,0.065352,0.094584,0.116546,0.134364,0.039151,...,0.463655,0.278641,0.379614,0.312638,0.000000,0.334841,0.000000,10,"[-12.17503067677975, 9.66687598974862]",0
1,POLYGON ((-11.262036602130534 8.16996774850322...,0.040457,0.061447,0.169076,0.306898,0.120875,-0.223646,0.152342,0.294298,0.129666,...,0.616264,0.346009,0.390615,0.341503,0.369959,0.428866,0.375383,10,"[-11.262036573388308, 8.160969908434629]",1
2,POLYGON ((-11.131377263585799 8.07272125519212...,-0.012378,0.025284,0.139634,0.081894,0.053295,-0.098647,0.120698,0.182608,0.013352,...,0.718562,0.351211,0.315646,0.441170,0.367702,0.386080,0.046991,10,"[-11.13137723485804, 8.06372341512419]",2
3,POLYGON ((-12.943973629787859 8.91735381448926...,-0.056811,0.018540,0.112298,0.100867,0.012564,0.091831,0.096314,0.243000,0.074746,...,0.491696,0.350578,0.294817,0.793215,0.366045,0.198691,0.771262,10,"[-12.94397360098245, 8.90835597442643]",3
4,POLYGON ((-10.642673866317843 8.54895470218197...,-0.034265,0.046373,0.098283,0.077979,0.037578,0.171010,0.121800,0.066459,0.083704,...,0.526977,0.317128,0.414979,0.361227,0.000000,0.387959,0.000000,10,"[-10.642673837548983, 8.5399568621107]",5


In [34]:
# drop extra columns 'Map' and 'geometry' if they exist
nighttime_trees_df = nighttime_trees_df.drop(columns=['Map', 'geometry'])
nighttime_water_df = nighttime_water_df.drop(columns=['Map', 'geometry'])

# set id column to be equal to the row number
nighttime_trees_df['id'] = nighttime_trees_df.index
nighttime_water_df['id'] = nighttime_water_df.index

nighttime_trees_df.head(5)


,20140101,20140201,20140301,20140401,20140501,20140601,20140701,20140801,20140901,20141001,...,20230301,20230401,20230501,20230601,20230701,20230801,20230901,20231001,center,id
0,-0.038235,0.138977,0.251203,0.094305,0.065352,0.094584,0.116546,0.134364,0.039151,0.077318,...,0.400553,0.463655,0.278641,0.379614,0.312638,0.000000,0.334841,0.000000,"[-12.17503067677975, 9.66687598974862]",0
1,0.040457,0.061447,0.169076,0.306898,0.120875,-0.223646,0.152342,0.294298,0.129666,-0.017323,...,0.378642,0.616264,0.346009,0.390615,0.341503,0.369959,0.428866,0.375383,"[-11.262036573388308, 8.160969908434629]",1
2,-0.012378,0.025284,0.139634,0.081894,0.053295,-0.098647,0.120698,0.182608,0.013352,0.037922,...,0.363829,0.718562,0.351211,0.315646,0.441170,0.367702,0.386080,0.046991,"[-11.13137723485804, 8.06372341512419]",2
3,-0.056811,0.018540,0.112298,0.100867,0.012564,0.091831,0.096314,0.243000,0.074746,0.032451,...,0.360806,0.491696,0.350578,0.294817,0.793215,0.366045,0.198691,0.771262,"[-12.94397360098245, 8.90835597442643]",3
4,-0.034265,0.046373,0.098283,0.077979,0.037578,0.171010,0.121800,0.066459,0.083704,0.008682,...,0.298686,0.526977,0.317128,0.414979,0.361227,0.000000,0.387959,0.000000,"[-10.642673837548983, 8.5399568621107]",4


In [36]:

# melt the data into long format
nighttime_trees_melt_df = nighttime_trees_df.melt(id_vars=['id', 'center'], var_name='image_date', value_name='image_value')
nighttime_water_melt_df = nighttime_water_df.melt(id_vars=['id', 'center'], var_name='image_date', value_name='image_value')

nighttime_trees_melt_df.head(5)

,id,center,image_date,image_value
0,0,"[-12.17503067677975, 9.66687598974862]",20140101,-0.038235
1,1,"[-11.262036573388308, 8.160969908434629]",20140101,0.040457
2,2,"[-11.13137723485804, 8.06372341512419]",20140101,-0.012378
3,3,"[-12.94397360098245, 8.90835597442643]",20140101,-0.056811
4,4,"[-10.642673837548983, 8.5399568621107]",20140101,-0.034265


In [37]:
# export the melted dataframes to csv
nighttime_trees_melt_df.to_csv('data/dark/nighttime_trees_melt_df.csv', index=False)
nighttime_water_melt_df.to_csv('data/dark/nighttime_water_melt_df.csv', index=False)

## Mess Around with the Data and Visualizations

In [ ]:
bumpeh_lat = 7.891338
bumpeh_lng = -11.904920

# create a ee.Feature from the lat/lng
bumpeh = ee.Geometry.Point(bumpeh_lng, bumpeh_lat)

# pull in Sentinel-2 imagery
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
                  .filter(ee.Filter.date('2022-01-01', '2023-11-01')) \
                  .filterBounds(bumpeh)
# filter out cloudy images
s2 = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
s2_img = s2.median()

# visualize
s2_vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
m.addLayer(s2_img, s2_vis, 'Sentinel-2')
m.addLayer(bumpeh, {'color': 'ff0000'}, 'Bumpeh', True)

# zoom map into Bumpeh
m.setCenter(bumpeh_lng, bumpeh_lat, 14)

m